In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import base64
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import json

Inititialization

In [2]:

def load_data(url, driver_path):
    SCROLL_DOWN = "window.scrollTo(0, document.body.scrollHeight);"
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    time.sleep(5)
    
    load_button = driver.find_element(By.XPATH, "//button[contains(@class, 'ipc-see-more__button')]")
    
    for i in range(0, 3):
        print("Scrolling down...")
        driver.execute_script(SCROLL_DOWN)
        time.sleep(2)
        load_button.click()
        time.sleep(10)
        
    ret = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    
    script_tag = ret.find('script', {'id': '__NEXT_DATA__'})
    script_content = script_tag.contents[0]
    data = json.loads(script_content)
        
    return ret, data

def get_review_data(TITLE_CODE):
    review_url = f'https://www.imdb.com/title/{TITLE_CODE}/reviews?spoiler=hide&sort=reviewVolume&dir=desc&ratingFilter=0'
    soup = BeautifulSoup(requests.get(review_url).content, 'html.parser')
    review = soup.find_all('a', class_='title')
    return [r.text for r in review]

def get_Name(HTML_CONTENT, movie_names):
    names = HTML_CONTENT.find_all(
        'h3',
        class_='ipc-title__text'
    )
    for mov in names:
        movie_names.append(mov.text)

def get_TitleCode(HTML_CONTENT, imdb_movie_title_code):
    href = HTML_CONTENT.find_all(
        'div',
        class_="ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-479faa3c-9 dkLVoC dli-title"
    )
    for href in href:
        imdb_movie_title_code.append(href.find('a')['href'].split('/')[2])

def get_ReleaseDate(HTML_CONTENT, movie_release_date):
    movie_detail = HTML_CONTENT.find_all(
        'div',
        class_='sc-479faa3c-7 jXgjdT dli-title-metadata'
    )
    for m in movie_detail:
        spans = m.find_all('span')
        movie_release_date.append(spans[0].text)

def get_Duration(HTML_CONTENT, movie_duration_time):
    movie_detail = HTML_CONTENT.find_all(
        'div',
        class_='sc-479faa3c-7 jXgjdT dli-title-metadata'
    )
    for m in movie_detail:
        spans = m.find_all('span')
        movie_duration_time.append(spans[1].text)

def get_Rating(HTML_CONTENT_MOV, movie_ratings):
    rating = HTML_CONTENT_MOV.find_all(
        'span',
        class_='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating'
    )
    for r in rating:
        movie_ratings.append(float(r.text[:3]))

def get_Metascore(HTML_CONTENT, movie_metascore):
    meta_rating = HTML_CONTENT.find_all(
        'div',
        class_='ipc-metadata-list-summary-item__tc'
    )
    for m in meta_rating:
        score = m.find(
            'span',
            class_='sc-b0901df4-0 bcQdDJ metacritic-score-box'
        )
        if score:
            movie_metascore.append(int(score.text))
        else:
            movie_metascore.append(np.nan)

def get_Review(HTML_CONTENT, imdb_movie_title_code, reviews):
    for i, j in zip(imdb_movie_title_code, range(0, len(imdb_movie_title_code))):
        reviews.append(get_review_data(i))
        print(f"Done for {j + 1}: ({i})")


In [3]:
DRIVER_PATH = "A:\chromedriver-win64\chromedriver.exe"
URL_MOVIES = 'https://www.imdb.com/search/title/?sort=user_rating,asc&groups=top_1000&count=250'
URL_TVSHOWS = 'https://www.imdb.com/search/title/?title_type=tv_series&sort=num_votes,desc&num_votes=1000,&languages=en&count=250'

HTML_CONTENT_MOV, json_data = load_data(URL_MOVIES, DRIVER_PATH)
# HTML_CONTENT_TV = load_data(URL_TVSHOWS, DRIVER_PATH)


Scrolling down...
Scrolling down...
Scrolling down...


In [ ]:
imdb_movie_title_code = []
movie_names = []
movie_reviews = []
movie_ratings = []
movie_release_date = []
movie_metascore = []
movie_duration_time = []
# movie_genres = []
# movie_directors = []
# movie_stars = []


get_TitleCode(HTML_CONTENT_MOV, imdb_movie_title_code)
get_Name(HTML_CONTENT_MOV, movie_names)
get_Rating(HTML_CONTENT_MOV, movie_ratings)
get_ReleaseDate(HTML_CONTENT_MOV, movie_release_date)
get_Metascore(HTML_CONTENT_MOV, movie_metascore)
get_Duration(HTML_CONTENT_MOV, movie_duration_time)
get_Review(HTML_CONTENT_MOV, imdb_movie_title_code, movie_reviews)



dataframe_mov = {
    'Name': movie_names[:-1],
    'Release Date': movie_release_date,
    'Duration': movie_duration_time,
    'Rating': movie_ratings,
    'Metascore': movie_metascore,
    'title_code': imdb_movie_title_code,
    'reviews': movie_reviews,
}

pd.DataFrame(dataframe_mov).to_excel('movies.xlsx', index=False)
